# Part 3: Hardware Timers

#### Learning goals:
* Basic knowledge of what a hardware timer is and what it does
* Familiarity with the most important configuration options determining hardware timer behavior

#### Support Literature

* [User manual STM32 Nucleo-64 boards](https://www.st.com/content/ccc/resource/technical/document/user_manual/98/2e/fa/4b/e0/82/43/b7/DM00105823.pdf/files/DM00105823.pdf/jcr:content/translations/en.DM00105823.pdf)
* [HAL documentation](Documents/um1725-description-of-stm32f4-hal-and-lowlayer-drivers-stmicroelectronics.pdf)
* [Reference Manual stm32f446xx](Documents/dm00135183-stm32f446xx-advanced-arm-based-32-bit-mcus-stmicroelectronics.pdf)

#### Introduction to hardware peripherals:

On a microcontroller such as the STM32, hardware peripherals are separate hardware components capable of performing specialized tasks in accordance with a set of instructions provided by the executing program. When programming in for instance Arduino C, the interaction with hardware peripherals is hidden by a layer of abstraction. This removes a lot of the complexity of programming a microcontroller, but also places constrains on how those peripheral components can be used. 

Sampling an analog signal at a uniform sampling rate is $f_s$ a very clear case where making full use of hardware peripherals can result in a more efficient (and likely reliable) system. When a microprocessor program uses functions such as [`analogRead()`](https://docs.arduino.cc/language-reference/en/functions/analog-io/analogRead/) to read from an analog input, it is up to the software to keep track of *when* to start the analog-to-digital conversions. By appropriate configuration of a Hardware Timer in conjunction with an Analog-to-Digital Converter(ADC) it is possible to offload the job of managing sampling times to these two hardware peripherals, and only requiring the CPU to intervene when the sampled data is ready for further processing. 

<!-- The configuration of hardware peripherals is - at the end of the day - accomplished by manipulating [*registers*], using reserved macros in the programming language to access individual bits in the registers. Luckily, many hardware providers suppliers (ST Microelectronics included) will often supply libraries which manage separate hardware components without requiring the programmer to implement all the fiddly register manipulations. 

this problem, we will cover some basic theory of how the two aforementioned hardware peripherals work, and how certain sets of configuration will allow for sampling a signal at a set sample rate.-->

#### Timer basics

Hardware timers are relatively simple digital circuits which are used to generate PWM signals, set the symbol rate for serial communication, and much more. It is controlled by a clock signal input (usually an internal clock signal), and can trigger an update event at specific time intervals. Very briefly explained, the main component is a *counting register* which can increment the binary number stored in the register by `1` for each clock cycle. Then, when the counting register reaches a pre-configured value stored in a separate `Auto Reload Register`, the timer generates an output, and all the bits in the counting register are set to `0` so the timer can begin counting up again. It is important to note that counting starts at `0` and stops at the`ARR` value, meaning the time between resets will be `ARR`$+1$ clock cycles.

In addition, it is possible to configure a clock prescaler to scale down the input clock signal to the timer. The prescaler can be any integer value, although once again the factor by which the clock signal is scaled will be `Prescaler` $+1$ (otherwise a prescaler value of $0$ would imply dividing the clock speed by $0$). The illustration below shows how a simplified hardware timer configuration converts a clock signal into a pulse every $6$ clock cycles.

![](Figures/Timer_illustration.png)

One highly relevant use for update event generated by a hardware timer is to trigger an A/D conversion, meaning the ADC will read one sample from the analog input each time it is triggered by the hardware timer's trigger event. Assuming the full ADC conversion time is lower than the time between timer trigger events, the ADC's sampling frequency can be calculated as follows:

$$f_s = \frac{\text{System Clock Speed}}{\text{Prescaler Register Value}+1}\cdot \frac{1}{\text{Auto Reload Register (ARR) Value } + 1} \tag{3.1}$$

It is recommended you refer to [this](https://wiki.st.com/stm32mcu/wiki/Getting_started_with_TIM) guide for more information on the STM32's timers.



## a) calculating register values
For this assignment we want to generate update events at a rate of $f_s = 10\ 000 \text{ Hz}$. Using default system settings for the STM32F446re, the peripheral clock controlling our timer will be $84 \text{ MHz}$. Based on equation $3.1$, propose a combination of Prescaler (PSC) and Auto Reload Register (ARR) values which accomplish this.

In [ ]:
PSC = "???"
ARR = "???"
# WRITE YOUR CODE IN THIS CELL:

In [ ]:
from hashlib import sha1
assert sha1(str(round(float((ARR+1)*(PSC+1)), 0)).encode('utf-8')+b'c76c6').hexdigest() == '3cde1bcf641e9b6ede76db35cc8c93c58bb408fc', 'ARR and PSC do not combine to produce the correct result:('
print('Correct answer :)')

## b) configuring hardware timer 2

For this assignment we will be using the STM32F446re's general-purpose timer Timer 2 to generate trigger events. In STM32 CubeIDE, go to the "Device Configuration Tool", and proceed with the following steps:
1. Select the "Clock Configuration" tab, and verify the `APB1 Timer clocks` bus is set to $84\ \text{MHz}$. 
2. Select the the "Pinout & Configuration" tool, select "Timers->TIM2" and modify the following fields:
    * Set the `Clock Source` to "Internal Clock"
    * Set `Channel1` as "PWM Generation CH1"
    * Adjust the `Prescaler (PSC - 16 bits value)` according to your answer in problem **a)**
    * Adjust the `Counter Period (AutoReload Register - 32 bits value)` according to your answer in problem **a)**
    * Trigger Event Selection: `Update Event`
    * To produce a $50\%$ duty cycle PWM signal on `CH1`, set the `Pulse (32 bits value)` to be half the AutoReload Register period
  

[Here](Figures/HwTimConfig.png) is a screenshot highlighting the configuration options we need to modify. You may also need to make sure that the parameter `

## c) add custom code

Save the project and click "Yes" when prompted to generate code. Then, add the following line to the `USER CODE BEGIN 2` section to start the timer:

>```C
>/* USER CODE BEGIN 2 */
>HAL_TIM_PWM_Start(&htim2, TIM_CHANNEL_1); // Start Timer 2 in PWM mode with channel 1 as output
>/* USER CODE END 2 */
>```

You can now compile the project, and run the program on the STM32

## d) monitor PWM signal

If the STM32 is programmed correctly, you should be able to monitor a $10 \text{ KHz}$, $50\%$ duty cycle square wave on one of the GPIO pins (you can see this in the graphical pinout view). A pinout overview for the F446re Nucleo Board is found on figure 24, chapter 7.10 in the [user manual](https://www.st.com/content/ccc/resource/technical/document/user_manual/98/2e/fa/4b/e0/82/43/b7/DM00105823.pdf/files/DM00105823.pdf/jcr:content/translations/en.DM00105823.pdf). Connect an oscilloscope to `CH1`, and verify that the measured signal is indeed a $10 \text{ KHz}$ $50\%$ duty cycle square wave. Take a snapshot of the oscillooscope screen with measurements highlighting the signal period, and paste it into the markdown cell below (if using Jupyter Lab).


<div class="alert alert-info">
<h4> Answer theory questions here!</h4>
</div>
